In [1]:
import requests
from typing import Union
import json


def authenticate(url: str, user: str, password: str):
    form_data = {
        "grant_type": "password",
        "username": user,
        "password": password
    }

    access_token = requests.post(
        f"{url}/auth/token", data=form_data).json()["access_token"]

    request_header = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    return access_token, request_header


def get_block(url: str, request_header: dict, block_identifier: Union[int, str], 
            full_transactions=False):
    
    params = {
        "block_identifier": block_identifier,
        "full_transactions": full_transactions,
    }

    response = requests.get(
        f"{url}/mainnet/history/block", params=params, headers=request_header)

    return response.json()


def get_transaction(url: str, request_header: dict, transaction_hash: str,
                    decode_input: bool = False):
    
    params = {
        "transaction_hash": transaction_hash,
        "decode_input": decode_input,
    }

    response = requests.get(
        f"{url}/mainnet/history/transaction", params=params, headers=request_header)

    return response.json()


def get_events(url: str, request_header: dict, contract_address: str, 
               from_block: Union[int, str] = 0, 
               to_block: Union[int, str] = "latest", 
               decode_events: bool = False):
    
    params = {
        "contract_address": contract_address,
        "from_block": from_block,
        "to_block": to_block,
        "decode_events": decode_events,
    }

    response = requests.get(
        f"{url}/mainnet/contract/events", params=params, headers=request_header)

    return response.json()



In [2]:
from dotenv import dotenv_values
config = dotenv_values("../.env")


In [3]:
url = config['DESS_NODE_URL']
user = config['DESS_NODE_USR']
password = config['DESS_NODE_PWD']

# Get access token and request header.
access_token, request_header = authenticate(url, user, password)


In [7]:
# Request block with number.
block_identifier = "latest"
full_transactions = False

block = get_block(url, request_header, block_identifier, full_transactions)

In [8]:
block

{'baseFeePerGas': 15298650342,
 'difficulty': 0,
 'extraData': '0x7273796e632d6275696c6465722e78797a',
 'gasLimit': 30000000,
 'gasUsed': 13677872,
 'hash': '0xb222d16177b7e58af7a92067eebfd64ec5fa890893bb27e32a1ccf9a258caada',
 'logsBloom': '0x152b4386ef0aaaee3ecea434b318a32b7930085b46157016b72b8775e683cac4226a916c848002d4c49df34e22268338027102309b207c85d245a65610ef75d34803f4c815c40c7eac87468a5dc9aaef2f65906480651a17bd920dd68978fa409b232c809273f30364cc1ad4206f3a7d98d84053eb1a1f214c520d50301d20c5fa5c82508e8635019048254832d20a22494e8145cf062aee506cb14a92b1a1399fa98b4938e56e674e43d3ef9a9aad203404a88a789198822890fe1a0854eb49bbea0cf764c30d060d001832f5253e0040c1630cf5ea3d1344bc880e40662575883f630889c4808b407f4c9038208530e8a1923859eafa408868d835b3067661',
 'miner': '0x1f9090aaE28b8a3dCeaDf281B0F12828e676c326',
 'mixHash': '0x96198eac82ce65499f2c6071bdd71a2766c7154479d1d96585b47f2774a89ec8',
 'nonce': '0x0000000000000000',
 'number': 18220532,
 'parentHash': '0xbcb69592fe857992d61291805796bb90

In [8]:
with open("../data/block_dess.json", "w") as f:
    json.dump(block, f, indent=4)


In [9]:
# Request single transaction with hash
transaction_hash = "0xd6d0b55a0d12f6d0a23da1d37e99969aca04af0311bd5d69095be7c9fb9b9756"
decode_input = False

transaction = get_transaction(
    url, request_header, transaction_hash, decode_input)

with open("../data/transaction_dess.json", "w") as f:
    json.dump(transaction, f, indent=4)


In [9]:
# request events of contract
contract_address = "0x889edC2eDab5f40e902b864aD4d7AdE8E412F9B1"
from_block = 18122866
to_block = "latest"
decode_events = False

events = get_events(url, request_header, contract_address,
                    from_block, to_block, decode_events)

with open("../data/events_dess.json", "w") as f:
    json.dump(events, f, indent=4)

In [10]:
len(events)

5532

In [21]:
limit = 100
txs = []
last_block_num = 18220551
for idx in range(limit):
    block_num = last_block_num - idx
    # res = w3.eth.get_block(block_num)
    block = get_block(url, request_header, block_num, True)
    for idx2, tx_details in enumerate(block['transactions']):
        txs.append([
            tx_details['from'],
            tx_details['to'],
            tx_details['value']
        ])


In [12]:
last_block_num = 18220551
block_num = last_block_num
block = get_block(url, request_header, block_num, True)


In [16]:
block['transactions']


[{'blockHash': '0xc96075afddcb0aa5a6f40a012c7cef3ccc0ac6d4d1db738a10d8a49268e34700',
  'blockNumber': 18220551,
  'from': '0x406b5707C7436720FE93bfa12f9751f872f4a56e',
  'gas': 232120,
  'gasPrice': 14382838701,
  'maxFeePerGas': 14384276984,
  'maxPriorityFeePerGas': 0,
  'hash': '0x0a5a42cc7ed4592c89f4a816e01da95896dbd1db1f14a8bdd2d9f6e674c0e688',
  'input': '0xe9383a6800000000000000000000000000000000000000000000000000851a1b180fa6680000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000044000000000000000000000000000000000000000000000000000000000000000e0000000000000000000000000000000000000000000000000000000000000030000000000000000000000000000000000000000000000000000000000000003600000000000000000000000000000000000000000000000000000004b2d7bc9c40000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000009bc8430b2577b5a268bb3c5c020000000000000000000000008571c129f335832f6bbc76d494